<a href="https://colab.research.google.com/github/awagler2/NODE/blob/main/scorecard_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

# Load the datasets
df_imputed = pd.read_csv('full_imputed_final.csv')
fos19 = pd.read_csv('FieldOfStudyData1718_1819_PP.csv')
fos20 = pd.read_csv('FieldOfStudyData1819_1920_PP.csv')
fos21 = pd.read_csv('FieldOfStudyData1920_2021_PP.csv')
hrsu=pd.read_csv('HRSU.csv')

<ipython-input-5-d3725dfcdb18>:4: DtypeWarning: Columns (26,29,30,501,502) have mixed types. Specify dtype option on import or set low_memory=False.
  df_imputed = pd.read_csv('full_imputed_final.csv')


In [11]:
print(fos19['UNITID'].value_counts())
print(fos20['UNITID'].value_counts())
print(fos21['UNITID'].value_counts())

UNITID
214777.0    536
201885.0    463
193900.0    453
174066.0    445
215293.0    430
           ... 
493567.0      1
493576.0      1
493585.0      1
493600.0      1
493628.0      1
Name: count, Length: 6689, dtype: int64
UNITID
495767.0    553
214777.0    523
201885.0    472
174066.0    449
104151.0    440
           ... 
496052.0      1
496292.0      1
451237.0      1
451255.0      1
451282.0      1
Name: count, Length: 6419, dtype: int64
UNITID
495767.0    551
201885.0    474
104151.0    467
104179.0    439
174066.0    436
           ... 
451167.0      1
451194.0      1
451237.0      1
451255.0      1
451282.0      1
Name: count, Length: 6255, dtype: int64


In [16]:
# Reshape fos19 to wide format
fos19w = fos19.pivot_table(index='UNITID', columns='CIPCODE', values='IPEDSCOUNT2')
fos20w = fos20.pivot_table(index='UNITID', columns='CIPCODE', values='IPEDSCOUNT2')
fos21w = fos21.pivot_table(index='UNITID', columns='CIPCODE', values='IPEDSCOUNT2')
# Display the first few rows of the restructured dataframe

print(fos19w.shape)
print(fos20w.shape)
fos21w.shape

(6255, 368)
(6158, 397)


(6072, 399)

In [34]:
# Merge the dataframes on the 'UNITID' column
merged_df = pd.merge(df_imputed, fos19, left_on='unitid',right_on='UNITID')
print(merged_df.columns)
print(df_imputed.shape)
print(fos19.shape)
print(merged_df.shape)
merged_df['UNITID'].value_counts

Index(['row_number', 'unitid', 'year_x', 'inst_name', 'longitude', 'latitude',
       'address', 'state_abbr', 'zip', 'phone_number',
       ...
       'EARN_COUNT_PELL_WNE_5YR', 'EARN_PELL_WNE_MDN_5YR',
       'EARN_COUNT_NOPELL_WNE_5YR', 'EARN_NOPELL_WNE_MDN_5YR',
       'EARN_COUNT_MALE_WNE_5YR', 'EARN_MALE_WNE_MDN_5YR',
       'EARN_COUNT_NOMALE_WNE_5YR', 'EARN_NOMALE_WNE_MDN_5YR',
       'EARN_COUNT_HIGH_CRED_5YR', 'EARN_IN_STATE_5YR'],
      dtype='object', length=684)
(23441, 510)
(228760, 174)
(1274636, 684)


<bound method IndexOpsMixin.value_counts of 0          100654.0
1          100654.0
2          100654.0
3          100654.0
4          100654.0
             ...   
1274631    133951.0
1274632    133951.0
1274633    133951.0
1274634    133951.0
1274635    133951.0
Name: UNITID, Length: 1274636, dtype: float64>

In [35]:
from IPython import get_ipython
from IPython.display import display
# %%
import pandas as pd

# Load the datasets
#df = pd.read_csv('ScoreCard_IPEDS.csv')
# Load the NSF_CIP_codes.csv and specify 'CIP Code' as string type
nsf=pd.read_csv('NSF_CIP_codes.csv', dtype={'CIP Code': str})

#nsf.head()
# %%
print(merged_df['CIPCODE'].head())
nsf['CIP2']=nsf['CIP Code']
# Now that 'CIP2' is guaranteed to be string type, .str accessor will work
nsf['CIP2'] = nsf['CIP2'].str.replace('.', '', regex=False)
merged_df['STEM_CIP'] = merged_df['CIPCODE'].apply(lambda x: x if str(x) in nsf['CIP Code'].values else 'Non-STEM') # Convert x to string for consistent comparison
merged_df['STEM'] = merged_df['CIPCODE'].apply(lambda x: 1 if str(x) in nsf['CIP2'].values else 0) # Convert x to string for consistent comparison

print(merged_df['STEM'].value_counts())
print(merged_df.shape)


0    100
1    109
2    110
3    110
4    110
Name: CIPCODE, dtype: int64
STEM
0    1021424
1     253212
Name: count, dtype: int64
(1274636, 686)


In [ ]:
import pandas as pd

hrsu=pd.read_csv('HRSU.csv')
#df=pd.read_csv('ScoreCard_IPEDS_CIP.csv')
#change to character strings
#hrsu['UNITID']=hrsu['UNITID'].astype(str)
#df['UNITID']=df['UNITID'].astype(str)
#check these are same type for comparison
print(hrsu['UNITID'].dtype)
print(merged_df['UNITID'].dtype)

# Create a binary variable 'HRSU' in d#f
merged_df['HRSU'] = merged_df['UNITID'].apply(lambda x: 1 if x in hrsu['UNITID'].values else 0)

# Print the value counts to check the result
print(merged_df['HRSU'].value_counts())
print(hrsu['UNITID'].values)

merged_df.to_csv('ScoreCard_IPEDS_CIP_HRSU.csv', index=False)


int64
float64
HRSU
0    1216095
1      95040
Name: count, dtype: int64
[104151 190576 133951 185828 228723 229115 228769 228778 228796 229027
 104179 110653 110671 110705 110714 132903 126562 225511 145600 182281
 187985 227216 122409 133669 188030 445188 110644]
